# Determinist Approach to the problem

In [1]:
%pip -q install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%load_ext autoreload
%autoreload 2

I. Explanation 

This is a determinist approch to the problem. There was no learning involve. Please consult ReadMe.md for more detail on implementation

II. Results : 

In [2]:
import simulate
from astaragent import MyAgent

eval_config_paths = [f"./EVAL/config_{i}.json" for i in range(1, 11)]
trained_agent = MyAgent()
all_results = simulate.evaluate(eval_config_paths, trained_agent)


--- Evaluating Configuration: ./EVAL/config_1.json ---
Episode 1/10, Step 29, Reward: 39.90, Evacuated: 4, Deactivated: 0
Episode 2/10, Step 24, Reward: 40.00, Evacuated: 4, Deactivated: 0
Episode 3/10, Step 25, Reward: 39.99, Evacuated: 4, Deactivated: 0
Episode 4/10, Step 22, Reward: 40.10, Evacuated: 4, Deactivated: 0
Episode 5/10, Step 26, Reward: 40.00, Evacuated: 4, Deactivated: 0
Episode 6/10, Step 24, Reward: 40.05, Evacuated: 4, Deactivated: 0
Episode 7/10, Step 23, Reward: 39.99, Evacuated: 4, Deactivated: 0
Episode 8/10, Step 34, Reward: 39.90, Evacuated: 4, Deactivated: 0
Episode 9/10, Step 22, Reward: 40.07, Evacuated: 4, Deactivated: 0
Episode 10/10, Step 31, Reward: 39.89, Evacuated: 4, Deactivated: 0

--- Evaluating Configuration: ./EVAL/config_2.json ---
Episode 1/10, Step 26, Reward: 39.91, Evacuated: 4, Deactivated: 0
Episode 2/10, Step 30, Reward: 39.82, Evacuated: 4, Deactivated: 0
Episode 3/10, Step 32, Reward: 39.75, Evacuated: 4, Deactivated: 0
Episode 4/10, St

In [3]:
display(all_results)

,config_path,episode,steps,reward,evacuated,deactivated
0,./EVAL/config_1.json,1,29,39.9012,4,0
1,./EVAL/config_1.json,2,24,39.9954,4,0
2,./EVAL/config_1.json,3,25,39.9908,4,0
3,./EVAL/config_1.json,4,22,40.0988,4,0
4,./EVAL/config_1.json,5,26,40.0026,4,0
...,...,...,...,...,...,...
95,./EVAL/config_10.json,6,69,-1644.2540,0,4
96,./EVAL/config_10.json,7,44,-863.2592,0,4
97,./EVAL/config_10.json,8,87,-2532.0535,1,3
98,./EVAL/config_10.json,9,102,-2924.1853,0,4


In [5]:
# Calculate averages for each configuration
averages = all_results.groupby('config_path').mean().reset_index().drop(columns=['episode'])
averages = averages.rename(columns={
    'steps': 'avg_steps',
    'reward': 'avg_reward',
    'evacuated': 'avg_evacuated',
    'deactivated': 'avg_deactivated'})

display(averages)
averages.to_csv('averages.csv', index=False)

,config_path,avg_steps,avg_reward,avg_evacuated,avg_deactivated
0,./EVAL/config_1.json,26.0,39.98976,4.0,0.0
1,./EVAL/config_10.json,59.5,-1477.06860,0.2,3.8
2,./EVAL/config_2.json,28.3,39.91543,4.0,0.0
3,./EVAL/config_3.json,37.8,-468.64999,1.5,2.5
4,./EVAL/config_4.json,49.9,-418.27644,2.4,1.6
5,./EVAL/config_5.json,51.7,-728.61907,1.5,2.5
6,./EVAL/config_6.json,58.6,-629.23686,1.5,2.5
7,./EVAL/config_7.json,84.1,-990.57649,1.8,2.2
8,./EVAL/config_8.json,51.4,-344.06660,1.3,2.7
9,./EVAL/config_9.json,38.6,-1000.69898,0.2,3.8
